<a href="https://colab.research.google.com/github/omarramy-74/Djikestra-AI-Agent/blob/main/Djikestra_Ai_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from langchain_together import ChatTogether
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_core.messages import SystemMessage
from pydantic import BaseModel

In [ ]:
%pip install langchain-together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 2.2 MB/s eta 0:00:00


In [ ]:
model = ChatTogether(model = "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
                     api_key = "tgp_v1_frWMlsoybz07FVkKvUwHnvuekBp2yQ7u7BX4Kl5bjd0")

In [ ]:
Answer = model.invoke("Do you know mohamed salah?")

In [ ]:
print(Answer.content)

I'm familiar with Mohamed Salah. He's a professional soccer player who currently plays as a forward for Liverpool FC in the English Premier League and the Egypt national team.

Mohamed Salah is widely regarded as one of the best players in the world, known for his exceptional speed, skill, and goal-scoring ability. He has won numerous awards and accolades, including the Premier League Golden Boot, the UEFA Champions League, and the African Footballer of the Year award.

Salah has also been recognized for his philanthropic efforts, particularly in his hometown of Nagrig, Egypt, where he has invested in various charitable projects, including a hospital, a school, and a sports facility.

He's a highly respected and beloved figure in the soccer world, and his achievements on and off the pitch have made him a role model for many young players and fans around the globe. Are you a fan of Mohamed Salah?


In [ ]:
from langchain_core.tools import tool
import heapq

@tool
def Djikestra(path: dict, start: str, end: str) -> dict:
    """
    A tool that finds the shortest path between two nodes using Dijkstra's Algorithm.

    Args:
        path (dict): A dictionary representing the graph in the format {node: [(neighbor, cost), ...]}
        start (str): The starting node.
        end (str): The destination node.

    Returns:
        dict: A dictionary with keys 'cost' and 'route', representing the shortest cost and path from start to end.
    """
    queue = [(0, start)]
    dist = {node: float('inf') for node in path}
    dist[start] = 0
    prev = {}

    while queue:
        cost, node = heapq.heappop(queue)
        if node == end:
            break
        for neighbor, weight in path.get(node, []):
            new_cost = cost + weight
            if new_cost < dist[neighbor]:
                dist[neighbor] = new_cost
                prev[neighbor] = node
                heapq.heappush(queue, (new_cost, neighbor))

    if end not in prev and start != end:
        return {"cost": None, "route": []} # Return None for cost and empty list for route

    route = []
    current = end
    while current != start:
        route.append(current)
        current = prev[current]
    route.append(start)
    route.reverse()

    return {"cost": dist[end], "route": route}

In [ ]:
Djikestra_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content = "You Are a min distance finder Agent and Expert at it"),
    ("system", "You have to make sure that the path dictionary is not empty and the start and end are provided"),
    ("system","Then you need to return the calc min path and the min cost and the moved path from the start to the end"),
    ("human","Find the min distance in this graph {graph}"),
    ("human","Start node is {start}"),
    ("human","End node is {end}"),
])


In [ ]:
%pip install langgraph

In [ ]:
from typing import Optional
from pydantic import BaseModel

class Djikestra_output_format(BaseModel):
    cost: Optional[str]
    route: Optional[list]

In [ ]:
import langgraph.prebuilt
Djikestra_agent = langgraph.prebuilt.create_react_agent(
    model = model,
    tools = [Djikestra],
    response_format=Djikestra_output_format,
)

In [ ]:
def Find_Min_path(graph: dict, start: int, end: int):
    print("Searcing for the min path")
    result = Djikestra_agent.invoke(Djikestra_prompt.invoke({"graph": graph, "start": start, "end": end}))
    return result

In [ ]:
#create a graph
graph = {
    "2" : [("1", 1)],
    "1" : [("2", 1), ("3", 1)],
    "3" : [("1", 1), ("4", 1)],
    "4" : [("3", 1), ("5", 1)],
}
start = "1"
end = "4"

In [ ]:
Result = Find_Min_path(graph, start, end)

Searcing for the min path


In [ ]:
print(Result['messages'][-1].content)

The minimum distance from node 1 to node 4 in the given graph is 2. The shortest path is 1 -> 3 -> 4.
